In [27]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_log_error,r2_score,mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb
import pickle

In [28]:
df=pd.read_csv('../vehiclesFinal.csv')
df.head()

,id,region,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,lat,long,price
0,7119256118,mohave county,2012.0,jeep,patriot,like new,4 cylinders,gas,247071.0,clean,automatic,4wd,full-size,offroad,silver,34.4554,-114.2690,3495
1,7120880186,oregon coast,2014.0,bmw,328i m-sport,good,5 cylinders,gas,76237.0,clean,automatic,rwd,full-size,sedan,grey,46.1837,-123.8240,13750
2,7115048251,greenville / upstate,2001.0,dodge,caravan,excellent,6 cylinders,gas,199000.0,clean,automatic,4wd,full-size,offroad,grey,34.9352,-81.9654,2300
3,7119250502,mohave county,2004.0,chevrolet,colorado ls,excellent,5 cylinders,gas,54000.0,clean,automatic,rwd,mid-size,pickup,red,34.4783,-114.2710,9000
4,7120430837,maine,2005.0,ford,mustang convertible,excellent,6 cylinders,gas,62800.0,clean,automatic,rwd,mid-size,convertible,silver,44.2070,-69.7858,8500


In [31]:
df2=df.copy()
df.drop(['id','title_status','lat','long','model','region'],axis=1,inplace=True)
df.head()

,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,size,type,paint_color,price
0,2012.0,jeep,like new,4 cylinders,gas,247071.0,automatic,4wd,full-size,offroad,silver,3495
1,2014.0,bmw,good,5 cylinders,gas,76237.0,automatic,rwd,full-size,sedan,grey,13750
2,2001.0,dodge,excellent,6 cylinders,gas,199000.0,automatic,4wd,full-size,offroad,grey,2300
3,2004.0,chevrolet,excellent,5 cylinders,gas,54000.0,automatic,rwd,mid-size,pickup,red,9000
4,2005.0,ford,excellent,6 cylinders,gas,62800.0,automatic,rwd,mid-size,convertible,silver,8500


In [32]:
df2[df2.year==2012.0].head()

,id,region,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,lat,long,price
0,7119256118,mohave county,2012.0,jeep,patriot,like new,4 cylinders,gas,247071.0,clean,automatic,4wd,full-size,offroad,silver,34.4554,-114.2690,3495
11,7119234657,mohave county,2012.0,nissan,titan pro-4x,fair,5 cylinders,gas,150000.0,clean,automatic,4wd,full-size,offroad,grey,34.5575,-114.3310,16988
32,7120716958,oregon coast,2012.0,acura,mdx,good,6 cylinders,gas,132867.0,clean,automatic,4wd,full-size,SUV,black,42.3899,-122.9220,12998
34,7120716942,oregon coast,2012.0,ram,pickup 1500,good,8 cylinders,gas,103492.0,clean,automatic,4wd,full-size,truck,black,42.3899,-122.9220,17998
43,7119872566,south florida,2012.0,cadillac,cts luxury sedan,good,6 cylinders,gas,49600.0,clean,automatic,fwd,mid-size,sedan,white,26.5747,-80.0794,12900


In [33]:
num_col=['year','odometer']
cat_cols=['manufacturer','condition','cylinders','fuel','transmission','drive','size','type','paint_color']

In [42]:
temp=[]
for i in cat_cols:
   temp.append(df[i].unique().flatten()) 

array(['jeep', 'bmw', 'dodge', 'chevrolet', 'ford', 'honda', 'toyota',
       'nissan', 'subaru', 'gmc', 'volkswagen', 'kia', 'acura', 'ram',
       'chrysler', 'hyundai', 'cadillac', 'volvo', 'mercedes-benz',
       'audi', 'infiniti', 'mazda', 'mini', 'buick', 'mitsubishi',
       'rover', 'pontiac', 'lincoln', 'lexus', 'fiat', 'jaguar',
       'mercury', 'saturn', 'tesla', 'harley-davidson', 'ferrari',
       'land rover', 'porche', 'alfa-romeo', 'morgan', 'aston-martin'],
      dtype=object)

In [7]:
df[cat_cols]=df[cat_cols].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374222 entries, 0 to 374221
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   year          374222 non-null  float64 
 1   manufacturer  374222 non-null  category
 2   condition     374222 non-null  category
 3   cylinders     374222 non-null  category
 4   fuel          374222 non-null  category
 5   odometer      374222 non-null  float64 
 6   transmission  374222 non-null  category
 7   drive         374222 non-null  category
 8   size          374222 non-null  category
 9   type          374222 non-null  category
 10  paint_color   374222 non-null  category
 11  price         374222 non-null  int64   
dtypes: category(9), float64(2), int64(1)
memory usage: 11.8 MB


In [8]:
columnsValue={}
for col in cat_cols:
    column_labels=list(range(1,len(df[col].unique())+1))
    rep={}
    for i,j in zip(column_labels,df[col].unique()):
        rep[j]=i
    columnsValue[col]=list(df[col].unique())
    df[col].replace(rep,inplace=True)

In [9]:
df.head()

,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,size,type,paint_color,price
0,2012.0,1,1,1,1,247071.0,1,1,1,1,1,3495
1,2014.0,2,2,2,1,76237.0,1,2,1,2,2,13750
2,2001.0,3,3,3,1,199000.0,1,1,1,1,2,2300
3,2004.0,4,3,2,1,54000.0,1,2,2,3,3,9000
4,2005.0,5,3,3,1,62800.0,1,2,2,4,1,8500


In [12]:
norm = StandardScaler()

norm.fit(df[['year','odometer']])
standardvalues=norm.transform(df[['year','odometer']])

df['year']=standardvalues[:,:1].flatten()
df['odometer']=standardvalues[:,1:].flatten()

pickle.dump(norm, open('StandardScaler.sav', 'wb'))

df['price'] = np.log(df['price'])
#scaling target variable
q1,q3=(df['price'].quantile([0.25,0.75]))
o1=q1-1.5*(q3-q1)
o2=q3+1.5*(q3-q1)
df=df[(df.price>=o1) & (df.price<=o2)]

In [13]:
df.head()

,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,size,type,paint_color,price
0,0.157474,1,1,1,1,2.611323,1,1,1,1,1,8.159089
1,0.537358,2,2,2,1,-0.403124,1,2,1,2,2,9.528794
2,-1.931885,3,3,3,1,1.763088,1,1,1,1,2,7.740664
3,-1.362060,4,3,2,1,-0.795506,1,2,2,3,3,9.104980
4,-1.172118,5,3,3,1,-0.640226,1,2,2,4,1,9.047821


In [14]:
X=df.iloc[:,:-1]
y=df['price']

xg_reg2 = xgb.XGBRegressor(objective ='reg:squarederror', learning_rate = 0.4,
                max_depth = 24, alpha = 5, n_estimators = 200)
xg_reg2.fit(X,y)
pickle.dump(xg_reg2, open('XGBoostDeploy.sav', 'wb'))

In [18]:
standardscaler= pickle.load(open('StandardScaler.sav', 'rb'))
mymodel = pickle.load(open('XGBoostDeploy.sav', 'rb'))

In [19]:
df.sample(2)

,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,size,type,paint_color,price
310730,-0.982176,32,4,2,1,0.532704,1,1,1,1,2,8.292799
78922,0.917242,14,4,2,1,0.388964,1,1,1,1,8,9.581559


In [20]:
df2.iloc[107333]

id                 7118842654
region          orange county
year                     2020
manufacturer           nissan
model             versa sedan
condition           excellent
cylinders         4 cylinders
fuel                      gas
odometer                 6052
title_status            clean
transmission        automatic
drive                     fwd
size                full-size
type                    sedan
paint_color            silver
lat                   33.8756
long                  -117.57
price                   13900
Name: 107333, dtype: object

In [45]:
year_odometer=pd.DataFrame(data=[[2020.0,6052]],columns=['year','odometer'])
x=standardscaler.transform(year_odometer[['year','odometer']]).flatten()

In [46]:
#get col uncoded value
df2.manufacturer.unique()[8-1]

'nissan'

In [47]:
test=pd.DataFrame(data=[[x[0],8,3,1,1,x[1],1,3,1,2,1]],columns=X.columns)
test

,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,size,type,paint_color
0,1.677009,8,3,1,1,-1.641571,1,3,1,2,1


In [ ]:
paint_color, manufacturer, year, transmission, cylinders, size, fuel, condition, drive, type, odometer

In [61]:
t=[1,8,x[0],1,1,1,1,3,3,2,x[1]]
t

[1, 8, 1.677008789434488, 1, 1, 1, 1, 3, 3, 2, -1.6415714056990034]

In [64]:
pred=mymodel.predict(test)
price=np.exp(pred[0])
print(price)

13605.137
